In [19]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [20]:
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [21]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.5, 0.5, 0.5], 
                                                            [0.5, 0.5, 0.5])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor()])


In [30]:
data_dir = '/home/santhosh/testProjects/cats_dogs'
train_dir = 'train'
test_dir = 'test'

train_data = datasets.ImageFolder(f'{data_dir}/{train_dir}/',transform=train_transforms)
test_data = datasets.ImageFolder(f'{data_dir}/{test_dir}/', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=32)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32)

In [31]:
model = models.densenet121(pretrained=True)

/home/santhosh/miniconda3/envs/py37_django/lib/python3.7/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [ ]:
model

## Freez the Features model

In [33]:
for param in model.parameters():
    param.requires_grad = False

As we can see the pretrained model has the classifier with input 1024 and output = 1000

```python
(classifier): Linear(in_features=1024, out_features=1000, bias=True)
```

For our classifier to work for dogs and cats we need to retrain only the classifier with 2 featurs (cats,dogs)

In [34]:
from collections import OrderedDict

classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(1024, 500)),
    ('relu1', nn.ReLU()),
    ('fc2', nn.Linear(500, 2)),
    ('output', nn.LogSoftmax(dim=1))]))

In [35]:
model.classifier = classifier

Now we are using the new classifier which is not trained. So our next step is to train our custom classifier.

In [36]:
import torch.optim as optim
import time

In [37]:
#for device in ['cpu','cuda']:
for device in ['cpu']:
    
    criterion = nn.NLLLoss()
    # Since we are training only our custom classifier, we need to
    # pass the classifier.parameters() to the optimizer.
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
    
    model.to(device)
    
    for ii, (inputs, labels) in enumerate(trainloader):
        
         # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)
        
    
        start = time.time()
        
        logPs = model.forward(inputs)
        loss = criterion(logPs, lables)
        
        loss.backward()
        optimizer.step()
        
        if ii == 3:
            break
            
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")
        

Device = cpu; Time per batch: 5.472 seconds
